In [ ]:
pip install selenium

In [131]:
import numpy as np
import pandas as pd

In [132]:
from selenium import webdriver

driver=webdriver.Chrome()

driver.get("https://worldemissions.io/") 

In [134]:
from selenium.webdriver.common.by import By
import time

## Sector
country_field = '//*[@id="SectorsHeader"]/div/div/input'
country = '//*[@id="SectorsHeader"]/div/div/div/div/p'
L1 = '//*[@id="sectors"]/article[2]/div[1]'
country_click = '//*[@id="SectorsHeader"]/div/div/ul/div/li/button/p'
country_click_congo = '//*[@id="SectorsHeader"]/div/div/ul/div/li[2]/button/p'
country_close = '//*[@id="SectorsHeader"]/div/div/div/img'

# countries = ["Indonesia", "Thailand", "Malaysia", "Viet Nam", "Philippines", "Singapore", "Cambodia", "Myanmar",
#             "Lao People's Democratic Republic", "Brunei Darussalam"]

countries = driver.find_element(By.XPATH, '//*[@id="SectorsHeader"]/div/div/ul').text.split('\n')[1:]

# Sub-sector
L2_data = '//*[@id="sectors"]/article[2]/aside/article[1]/div[2]/ul'
close = '//*[@id="sectors"]/article[2]/aside/article[1]/button'
L2_L1sector = '//*[@id="sectors"]/article[2]/aside/article[1]/div[1]/div[1]/h4'

L2_agri_click = '//*[@id="sectors"]/article[2]/div[2]/div[1]/button/span[2]'
L2_building_click ='//*[@id="sectors"]/article[2]/div[2]/div[2]/button/span[2]'
L2_energy_click = '//*[@id="sectors"]/article[2]/div[2]/div[3]/button/span[2]'
L2_industry_click = '//*[@id="sectors"]/article[2]/div[2]/div[4]/button/span[2]'
L2_transport_click = '//*[@id="sectors"]/article[2]/div[2]/div[5]/button/span[2]'

L2_clicks = [L2_agri_click, L2_building_click, L2_energy_click, L2_industry_click, L2_transport_click]

# Year slider
yslider = driver.find_element(By.XPATH,'//*[@id="root"]/main/section[6]/div/div[2]/div[3]/div/div/button')
y2022 = driver.find_element(By.XPATH, '//*[@id="root"]/main/section[6]/div/div[2]/div[3]/div/ul/li[3]')
y2030 = driver.find_element(By.XPATH, '//*[@id="root"]/main/section[6]/div/div[2]/div[3]/div/ul/li[11]')
year = driver.find_element(By.XPATH, '//*[@id="root"]/main/section[6]/div/div[2]/div[3]/div/div/span')

In [130]:
# setting up df
sectors =  ['Agriculture', 'Building', 'Energy', 'Industry', 'Transport']
L1_df = pd.DataFrame(columns = ['Year', 'Country', 'Sector', 'Emissions'])
L2_df = pd.DataFrame(columns = ['Year', 'Country', 'L1 Sector', 'L2 Sector','Emissions'])

#select years to scrape
years = [y2030]

for y in years:
    webdriver.ActionChains(driver).drag_and_drop(yslider, y).perform()
    current_year = year.text
    for i in countries:
        driver.find_element(By.XPATH, country_field).send_keys(i)
        time.sleep(2)
        ## exception for Congo since there are 2 Congos (but need to fix Guinea manually)
        if len(driver.find_element(By.XPATH, '//*[@id="SectorsHeader"]/div/div/ul').text.split('\n'))==1:
            driver.find_element(By.XPATH, country_click).click()
            time.sleep(3)
            aa = driver.find_element(By.XPATH, country).text
            aaa = driver.find_element(By.XPATH, L1).text
            aaaa = aaa.split('\n')
            print(aa)
            print(aaaa)
            temp_df = pd.DataFrame(
                {'Year': current_year, 'Country': aa,
                'Sector': sectors,
                'Emissions': aaaa}
            )
            L1_df = L1_df.append(temp_df, ignore_index=True)
        else:
            driver.find_element(By.XPATH, country_click_congo).click()
            time.sleep(1.5)
            aa = driver.find_element(By.XPATH, country).text
            aaa = driver.find_element(By.XPATH, L1).text
            aaaa = aaa.split('\n')
            print(aa)
            print(aaaa)
            temp_df = pd.DataFrame(
                {'Year': current_year, 'Country': aa,
                'Sector': sectors,
                'Emissions': aaaa}
            )
            L1_df = L1_df.append(temp_df, ignore_index=True)

        for j in L2_clicks:
            driver.find_element(By.XPATH, j).click()
            time.sleep(1.5)
            asd = driver.find_element(By.XPATH, L2_data).text.split('\n')
            L2_sectors = asd[::2]
            L2_emissions = asd[1::2]
            if L2_sectors != ['']:
                temp_df = pd.DataFrame(
                    {'Year': current_year, 'Country': driver.find_element(By.XPATH, country).text,
                    'L1 Sector': driver.find_element(By.XPATH, L2_L1sector).text,
                     'L2 Sector': L2_sectors,
                    'Emissions': L2_emissions})
                L2_df = L2_df.append(temp_df, ignore_index=True)
                driver.find_element(By.XPATH, close).click()
                time.sleep(1.5)
            else:
                driver.find_element(By.XPATH, close).click()
                time.sleep(2)
        driver.find_element(By.XPATH, country_close).click()

L1_df.to_csv(r"C:\Users\58706\Bain\Haran, Hari - 2023_Bayshore_Climate Strategy\6. Workstreams\1. GHG emission landscaping\World Emissions Clock\Output\World BAU\L1 - 2030.csv")
L2_df.to_csv(r"C:\Users\58706\Bain\Haran, Hari - 2023_Bayshore_Climate Strategy\6. Workstreams\1. GHG emission landscaping\World Emissions Clock\Output\World BAU\L2 - 2030.csv")

Aruba
['2.6 KT', '95.3 KT', '697.3 KT', '186.1 KT', '614.9 KT']
Afghanistan
['17.7 MT', '374.3 KT', '4.9 MT', '13.4 MT', '3.0 MT']
Angola
['65.1 MT', '5.6 MT', '25.4 MT', '17.4 MT', '11.8 MT']
Albania
['19.4 KT', '429.1 KT', '253.3 KT', '3.9 MT', '2.4 MT']
United Arab Emirates
['-7.0 MT', '2.4 MT', '153.7 MT', '155.7 MT', '44.6 MT']
Argentina
['234.7 MT', '35.4 MT', '121.1 MT', '74.1 MT', '66.6 MT']
Armenia
['1.7 MT', '2.8 MT', '2.7 MT', '1.5 MT', '3.6 MT']
Australia
['62.1 MT', '16.2 MT', '236.7 MT', '99.4 MT', '106.6 MT']
Austria
['5.5 MT', '7.2 MT', '16.5 MT', '26.3 MT', '26.1 MT']
Azerbaijan
['8.9 MT', '10.5 MT', '23.0 MT', '11.2 MT', '11.4 MT']
Burundi
['7.1 MT', '763.2 KT', '774.5 KT', '2.9 MT', '496.6 KT']
Belgium
['8.2 MT', '19.1 MT', '25.5 MT', '36.9 MT', '24.4 MT']
Benin
['18.4 MT', '1.5 MT', '2.3 MT', '8.1 MT', '11.3 MT']
Burkina Faso
['52.6 MT', '2.5 MT', '3.3 MT', '10.7 MT', '4.4 MT']
Bangladesh
['150.8 MT', '20.8 MT', '125.1 MT', '112.2 MT', '33.1 MT']
Bulgaria
['-3.9 MT'

Romania
['-15.2 MT', '11.7 MT', '24.8 MT', '30.8 MT', '24.5 MT']
Pakistan
['318.9 MT', '45.7 MT', '94.4 MT', '194.3 MT', '75.9 MT']
Panama
['8.6 MT', '762.1 KT', '4.9 MT', '7.4 MT', '6.2 MT']
Peru
['233.7 MT', '3.9 MT', '31.0 MT', '26.7 MT', '30.2 MT']
Philippines
['74.2 MT', '18.6 MT', '146.1 MT', '75.4 MT', '41.2 MT']
Papua New Guinea
['79.3 MT', '574.1 KT', '7.6 MT', '2.7 MT', '4.2 MT']
Poland
['36.7 MT', '35.4 MT', '154.9 MT', '59.5 MT', '89.2 MT']
Puerto Rico
['728.6 KT', '226.1 KT', '28.2 KT', '2.6 MT', '21.3 KT']
Portugal
['2.1 MT', '3.1 MT', '16.0 MT', '18.7 MT', '17.0 MT']
Paraguay
['89.4 MT', '699.9 KT', '2.0 MT', '4.1 MT', '11.1 MT']
Qatar
['756.0 KT', '490.7 KT', '137.2 MT', '50.7 MT', '11.9 MT']
Romania
['-15.2 MT', '11.7 MT', '24.8 MT', '30.8 MT', '24.5 MT']
Russian Federation
['31.1 MT', '218.3 MT', '1.1 GT', '480.2 MT', '258.2 MT']
Rwanda
['7.1 MT', '728.7 KT', '450.8 KT', '3.4 MT', '656.3 KT']
Saudi Arabia
['-6.6 MT', '9.8 MT', '535.6 MT', '421.4 MT', '179.0 MT']
Sudan

In [138]:
# manual corrections
L1_df_manual = pd.DataFrame(columns = ['Year', 'Country', 'Sector', 'Emissions'])
L2_df_manual = pd.DataFrame(columns = ['Year', 'Country', 'L1 Sector', 'L2 Sector','Emissions'])
manual_countries = ["Guinea", "Niger", "Oman"]

#select years to scrape
years = [y2030]

for y in years:
    webdriver.ActionChains(driver).drag_and_drop(yslider, y).perform()
    current_year = year.text
    for i in manual_countries:
        driver.find_element(By.XPATH, country_field).send_keys(i)
        time.sleep(2)
        driver.find_element(By.XPATH, country_click).click()
        time.sleep(3)
        aa = driver.find_element(By.XPATH, country).text
        aaa = driver.find_element(By.XPATH, L1).text
        aaaa = aaa.split('\n')
        print(aa)
        print(aaaa)
        temp_df = pd.DataFrame(
                    {'Year': current_year, 'Country': aa,
                    'Sector': sectors,
                    'Emissions': aaaa}
                )
        L1_df_manual = L1_df_manual.append(temp_df, ignore_index=True)

        for j in L2_clicks:
            driver.find_element(By.XPATH, j).click()
            time.sleep(1.5)
            asd = driver.find_element(By.XPATH, L2_data).text.split('\n')
            L2_sectors = asd[::2]
            L2_emissions = asd[1::2]
            if L2_sectors != ['']:
                temp_df = pd.DataFrame(
                    {'Year': current_year, 'Country': driver.find_element(By.XPATH, country).text,
                    'L1 Sector': driver.find_element(By.XPATH, L2_L1sector).text,
                     'L2 Sector': L2_sectors,
                    'Emissions': L2_emissions})
                L2_df_manual = L2_df_manual.append(temp_df, ignore_index=True)
                driver.find_element(By.XPATH, close).click()
                time.sleep(1.5)
            else:
                driver.find_element(By.XPATH, close).click()
                time.sleep(2)
        driver.find_element(By.XPATH, country_close).click()

L1_df_manual.to_csv(r"C:\Users\58706\Bain\Haran, Hari - 2023_Bayshore_Climate Strategy\6. Workstreams\1. GHG emission landscaping\World Emissions Clock\Output\World BAU\L1 - 2030_manual.csv")
L2_df_manual.to_csv(r"C:\Users\58706\Bain\Haran, Hari - 2023_Bayshore_Climate Strategy\6. Workstreams\1. GHG emission landscaping\World Emissions Clock\Output\World BAU\L2 - 2030_manual.csv")

Guinea
['39.6 MT', '2.3 MT', '2.6 MT', '8.0 MT', '1.9 MT']
Niger
['55.9 MT', '2.2 MT', '2.3 MT', '7.6 MT', '2.2 MT']
Oman
['2.3 MT', '22.2 MT', '83.7 MT', '45.9 MT', '17.5 MT']
